In [1]:
# import re
# from urllib.parse import unquote, unquote_plus
# import json
# from preprocess import *
# import pandas as pd
# from prepare_view_variant import *

# files = ['../data/v2/foxnews_content_v2.json', '../data/v2/nyt_content_isaac.json',
#              '../data/v2/nyt_content_liam.json', '../data/v2/nyt_content_sagar.json']
# clean_text = [remove_char_encoding, remove_special_char, make_lowercase]
# filepath = '../data/v2/nytfox_bias_collate.json'
# collate_data(files, save_to=filepath, clean_text=clean_text)

# train_content, train_title, test_content, test_title = train_test_split(input_file=filepath, test_split=0.80, shuffle=False)
# (content_vocab, content_word_index, content_index_word,
#     title_vocab, title_word_index, title_index_word) = vectorize_data(train_content, train_title)
# glove_index = build_glove_embed_index()

# bbc_content_emb = create_embeddings(train_content, glove_index, 256, 100, 'train_content')
# bbc_title_emb = create_embeddings(train_title, glove_index, 16, 100, 'train_title')
# bbc_title_tokens = create_token_labels(train_title, TITLE_VECTORIZER, dataset_name='train')

In [2]:
# import necessary packages 
import tensorflow as tf
import pickle

from transformer import TransformerModel

In [3]:
# get train content data from pickle 
with open('../data/embeddings/train_content_embeddings.pkl','rb') as f:
    train_content_emb = pickle.load(f)

# get train title data from pickle 
with open('../data/embeddings/train_title_embeddings.pkl','rb') as f:
    train_title_emb = pickle.load(f)

# get train title labels data from pickle 
with open('../data/embeddings/train_title_labels.pkl','rb') as f:
    train_title_tokens = pickle.load(f)


In [4]:
# create datasets for training and testing 
train_content_data = tf.convert_to_tensor(train_content_emb)
train_title_data = tf.convert_to_tensor(train_title_emb)
train_title_labels = tf.convert_to_tensor(train_title_tokens)

In [5]:
# Custom learning rate scheduler

# class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
#   def __init__(self, d_model, warmup_steps=4000):
#     super().__init__()

#     self.d_model = d_model
#     self.d_model = tf.cast(self.d_model, tf.float32)

#     self.warmup_steps = warmup_steps

#   def __call__(self, step):
#     step = tf.cast(step, dtype=tf.float32)
#     arg1 = tf.math.rsqrt(step)
#     arg2 = step * (self.warmup_steps ** -1.5)

#     return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
  
#   def get_config(self):
#     config = {'d_model':self.d_model,
#               'warmup_steps':self.warmup_steps}
#     return config

# learning_rate = CustomSchedule(d_model=100)

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)

In [6]:
# initalize model
model = TransformerModel(emb_sz=100, num_heads=5, key_dim=64, vocab_size=15000)

# compile model with Adam optimizer and masked loss and masked accuracy 
model.compile(optimizer='Adam', loss=TransformerModel.masked_loss, metrics=[TransformerModel.masked_accuracy])

# train model with token titles as labels 
model.fit(x=(train_content_data, train_title_data[:,:-1]), y=train_title_labels[:,1:], batch_size=200, epochs=1) # try 75 epochs

2023-05-09 17:16:04.270854: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


126/126 [==============================] - 232s 2s/step - loss: 7.9539 - masked_accuracy: 0.0265


In [7]:
# save model or weights
# model.save('../models/2blocks-datav2_1epoch') # remember to change name
model.save_weights('../weights/2blocks-fixedlr-datav2_1epoch_weights') # remember to change name


In [8]:
# from preprocess import create_embeddings, build_glove_embed_index
# import json

# glove_index = build_glove_embed_index()

# with open('../data/v2/bbc_neutral.json') as f:
#     data = json.load(f)

# # separate content and title data into separate lists
# bbc_test_content = [item['content'] for item in data]
# bbc_test_title = [item['title'] for item in data]

# bbc_test_content_emb = create_embeddings(bbc_test_content, glove_index, 256, 100, 'test_content')
# bbc_test_title_emb = create_embeddings(bbc_test_title, glove_index, 16, 100, 'test_title')

In [9]:
# import os
# model_name = '../models/2blocks-customlr-datav2'

# if os.file.exists(model_name):
#     temp = input("Model already exists. Press y to overwrite: ")
#     if temp == 'y':
#         model.save(model_name)
#     else:
#         print("Model not saved")
# else:
#     model.save(model_name)